# Setup data

In [ ]:
!gdown --id 1f6kICQBAMKaJ11qxEKduzbVBiK1e3VK7

In [ ]:
!unzip data_detect_xml_ltwh_80_20.zip

In [ ]:
!gdown --id 1FeHeDko4RCsRfzzAIZx1O7VRxE7q6tFB

In [ ]:
!mkdir dataset
%cd dataset
!mkdir images
!mkdir labels
%cd ..

In [28]:
import glob2
import os
import cv2
from shutil import copyfile
from tqdm import tqdm

def move_folder(dir):
  new_image_dir = os.path.join("dataset", "images", dir)
  new_label_dir = os.path.join("dataset", "labels", dir + "_tlwh")
  
  os.mkdir(new_image_dir)
  os.mkdir(new_label_dir)

  list_image_path = glob2.glob(dir + "/*.jpg")
  
  with tqdm(total=len(new_image_dir)) as pbar:
    for image_path in list_image_path:
      image = cv2.imread(image_path)
      old_label_path = image_path.split(".")[0] + ".xml"
      new_label_path = os.path.join(new_label_dir, (image_path.split("/")[-1]).split(".")[0] + ".xml")
      new_image_path = os.path.join(new_image_dir, image_path.split("/")[-1])
      
      # move 
      cv2.imwrite(new_image_path, image)
      copyfile(old_label_path, new_label_path)

      pbar.update(1)


In [ ]:
# move_folder("val")
# move_folder("train")

# Convert tlwh_xml -> coco maxmin.

In [32]:
list_label_train =  glob2.glob("dataset/labels/train_tlwh/*.xml")

['dataset/labels/train_tlwh/cam_07_285.xml', 'dataset/labels/train_tlwh/data_1061.xml', 'dataset/labels/train_tlwh/data_3282.xml', 'dataset/labels/train_tlwh/cam_01_427.xml', 'dataset/labels/train_tlwh/data_420.xml', 'dataset/labels/train_tlwh/cam_01_246.xml', 'dataset/labels/train_tlwh/data_3507.xml', 'dataset/labels/train_tlwh/cam_14_88.xml', 'dataset/labels/train_tlwh/cam_01_277.xml', 'dataset/labels/train_tlwh/data_1637.xml', 'dataset/labels/train_tlwh/data_1332.xml', 'dataset/labels/train_tlwh/data_416.xml', 'dataset/labels/train_tlwh/data_3262.xml', 'dataset/labels/train_tlwh/data_2947.xml', 'dataset/labels/train_tlwh/data_3594.xml', 'dataset/labels/train_tlwh/data_353.xml', 'dataset/labels/train_tlwh/data_1962.xml', 'dataset/labels/train_tlwh/cam_07_210.xml', 'dataset/labels/train_tlwh/data_1452.xml', 'dataset/labels/train_tlwh/data_930.xml', 'dataset/labels/train_tlwh/data_123.xml', 'dataset/labels/train_tlwh/data_1981.xml', 'dataset/labels/train_tlwh/data_513.xml', 'dataset/la

In [38]:
with open("label_train_path.txt", "a+") as f:
  for path in list_label_train:
    f.write("{}\n".format(path))

In [42]:
list_label_val =  glob2.glob("dataset/labels/val_tlwh/*.xml")

In [44]:
with open("label_val_path.txt", "a+") as f:
  for path in list_label_val:
    f.write("{}\n".format(path))

In [45]:
classes = ["xe_may", "xe_ba_gac", "xe_taxi", "xe_hoi", "xe_ban_tai", "xe_cuu_thuong", "xe_khach", "xe_buyt", "xe_tai", "xe_container"]
with open("classes.txt", "a+") as f:
  for my_class in classes:
    f.write("{}\n".format(my_class))

# Convert voc2coco

Link file: [voc2coco](https://github.com/anminhhung/voc2coco/blob/master/voc2coco.py)

```python3
  # format xml : max min
  # xmax = int(bndbox.findtext('xmax'))
  # ymax = int(bndbox.findtext('ymax'))

  # format xml: tlwh
  width = int(bndbox.findtext('width'))
  height = int(bndbox.findtext('height'))
  xmax = xmin + width
  ymax = ymin + height
```

Nếu sử dụng max min thì cmt lại :p vẫn còn hardcode khúc này

In [50]:
!python voc2coco.py \
    --ann_paths_list label_train_path.txt \
    --labels classes.txt \
    --output instances_train.json \

Start converting !
100% 5926/5926 [00:00<00:00, 10969.44it/s]


In [ ]:
!python voc2coco.py \
    --ann_paths_list label_val_path.txt \
    --labels classes.txt \
    --output instances_val.json \

In [52]:
!mkdir dataset/annotations

In [53]:
!mv instances_val.json dataset/annotations
!mv instances_train.json dataset/annotations
!mv classes.txt dataset/annotations

# Git clone

In [ ]:
!git clone https://github.com/anminhhung/Object_detection

In [55]:
!mv dataset Object_detection/EfficientDet/

In [69]:
!mv dataset/images/train dataset/train
!mv dataset/images/val dataset/val

In [71]:
!rm -rf dataset/images

In [ ]:
%cd Object_detection/EfficientDet/

# Config


In [ ]:
# Use format 2
'''
CONFIG:
    # setup dataset
    ROOT_DIR: "."
    LABELS_PATH: "dataset/annotations/classes.txt"
    TRAIN_DIR: "dataset"
    IMG_TRAIN_DIR: "."
    SET_TRAIN_DIR: "train"
    VAL_DIR: "dataset"
    IMG_VAL_DIR: "."
    SET_VAL_DIR: "val"
    
    # Avalabel models
    MODEL_NAME: "efficientdet-d0.pth"
    
    # hyper params
    NUM_GPUS: 1
    FREEZE_HEAD: False
    OPTIMIZER: sgd
    LR: 0.000001
    ES_MIN_DELTA: 0.0
    ES_PATIENCE: 0
    BATCH_SIZE: 4
    NUM_WORKERS: 4
    NUM_EPOCHS: 20
    VAL_INTERVAL: 1
    SAVE_INTERVAL: 1
'''

# Train

In [ ]:
!cd installation/ && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

In [ ]:
!python3 train.py